Cargamos las librerías, el conjunto de datos y algunas variables que emplearemos durante el proceso:

In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

df = pd.read_csv('./processed_data/houses_integrated.csv')

# Definimos las opciones de visualizacion
pd.set_option('display.max_columns', 500)

# Definimos un listado de colores para visualizaciones
clr = {'pr': '#5F66A1', 'yw': '#f3ca75',
       'mg': '#c874b9', 'gn': '#599d70', 'br': '#636261'}
b = '\033[94m'  # para texto azul
o = '\033[93m'  # para texto naranja
n = '\033[0m'   # para texto normal

# Omitimos los warnings
warnings.filterwarnings('ignore')

En esta fase verificaremos la calidad de los datos por medio de una serie de procesos.

# Estandarización de variables

## Estudio de variables a estandarizar

En este apartado se procede a estandarizar las variables, corrigiendo su tipo, y normalizando sus valores, (entendiendo por normalizar, asignarles el valor más representativo).

*NOTA: En este apartado no se tratarán ni los missing values ni los outliers. Su gestión se deja para más adelante.*

Empezaremos analizando cómo han sido asignadas las variables:

In [ ]:
df.dtypes

De la lista superior, extraemos qué variables estudiaremos en la estandarización. En concreto, nos centraremos en 4 grupos: 
+ Por un lado estandarizaremos las variables numéricas que no hayan asignado como tal
+ Por otro lado, estudiaremos las variables categóricas
+ A continuación estudiaremos las variables categoricas binarias, es decir, booleanas
+ Finalmente, nos centraremos en las variables de texto ligadas a la ubicación

<br />

**VARIABLES NUMÉRICAS NO ASIGNADAS COMO TAL**

+ <font color=#5F66A1>*bath_num*<font>
+ <font color=#5F66A1>*room_num*<font>
+ <font color=#5F66A1>*garage*<font>


**VARIABLES CATEGÓRICAS**

+ <font color=#5F66A1>*condition*<font>
+ <font color=#5F66A1>*energetic_certif*<font>
+ <font color=#5F66A1>*floor*<font>
+ <font color=#5F66A1>*heating*<font>
+ <font color=#5F66A1>*house_type*<font>
+ <font color=#5F66A1>*orientation*<font>


**VARIABLES BOOLEANAS**

+ <font color=#5F66A1>*air_conditioner*<font>
+ <font color=#5F66A1>*balcony*<font>
+ <font color=#5F66A1>*built_in_wardrobe*<font>
+ <font color=#5F66A1>*chimney*<font>
+ <font color=#5F66A1>*garden*<font>
+ <font color=#5F66A1>*kitchen*<font>
+ <font color=#5F66A1>*lift*<font>
+ <font color=#5F66A1>*reduced_mobility*<font>
+ <font color=#5F66A1>*storage_room*<font>
+ <font color=#5F66A1>*swimming_pool*<font>
+ <font color=#5F66A1>*terrace*<font>
+ <font color=#5F66A1>*unfurnished*<font>


**VARIABLES DE TEXTO LIGADAS A LA UBICACIÓN**

+ <font color=#5F66A1>*loc_zone*<font>
+ <font color=#5F66A1>*loc_district*<font>
+ <font color=#5F66A1>*loc_city*<font>
+ <font color=#5F66A1>*loc_neigh*<font>

## Estandarización de variables numéricas

### <font color=#5F66A1>bath_num</font>

In [ ]:
print(df.bath_num.unique())

Vemos como la ausencia de baños se identifica como 'sin baños', en forma de texto. Lo corregimos, y definimos la variable como numérica:

In [ ]:
df['bath_num'] = (df['bath_num']
                  .str.replace('sin baños','0')
                  .astype('int64')
                 )

### <font color=#5F66A1>room_num</font>

In [ ]:
print(df.room_num.unique())

Aquí tambien la ausencia de habitaciones se identifica en forma de texto, como 'sin habitación'. Lo corregimos, y definimos la variable como numérica:

In [ ]:
df['room_num'] = (df['room_num']
                   .str.replace('sin habitación','0')
                   .astype('int64')
                  )

### <font color=#5F66A1>garage</font>

In [ ]:
print(df.garage.unique()[:10])

Vemos que la variable refleja el precio de la plaza de garaje en forma de texto. Por lo tanto, extraemos el precio del texto y establecemos que la variable sea numérica:

In [ ]:
df['garage'] = (df['garage']
                .str.replace('plaza de garaje incluida en el precio','0')
                .str.replace('.','')
                .str.extract('(\d{1,4})')
                .astype('float64')
               )

## Estandarización de variables categóricas

### <font color=#5F66A1>condition</font>

In [ ]:
print(df.condition.unique())

Vemos que hay 3 posibles categorías. Redefinimos los nombres de las categorías para mejorar su interpretabilidad.

In [ ]:
df = df.replace({'condition':
                 {
                  'segunda mano/buen estado':'buen estado',
                  'segunda mano/para reformar':'para reformar',
                  'promoción de obra nueva':'obra nueva'
                 }})

df['condition'] = df['condition'].astype('category')

### <font color=#5F66A1>energetic_certif</font>

In [ ]:
print(df.energetic_certif.unique())

No requiere ningun tratamiento salvo definirla como categórica:

In [ ]:
df['energetic_certif'] = df['energetic_certif'].astype('category')

### <font color=#5F66A1>floor</font>

In [ ]:
print(df.floor.unique()[:15])

Vemos que, a grandes rasgos, esta variable esconde lo que podrían ser 3 variables distintas:

+ Número de planta en la que se encuentra el domicilio
+ Indicador de si la planta está en el interior o en el exterior
+ Número de plantas de las que consta el domicilio

Por lo tanto, se procede a dividir la variable en 3. 

<br />

Empezamos generando la variable booleana <font color=#5F66A1>__indoor__</font>, que tomará el valor True si el domicilio está en el interior, y False en el caso contrario:

In [ ]:
df['indoor'] = (df['floor']
                .str.contains('interior')
                .astype('category')   # no se define como bool porque aún contiene NaN
               )

print(df['indoor'].unique())

<br />

A continuación crearemos la variable <font color=#5F66A1>__floor_count__</font>, que recogerá el número de plantas de las que consta la casa.

Para ello, empezamos asignando como valor 1 a todas las casas que tengan información sobre la planta (las que estan como NaN las dejaremos por ahora tal cual, puesto que no tenemos información al respecto)

In [ ]:
df['floor_count'] = np.nan
df['floor_count'][df.floor.notna()] = 1

A continuación, añadiremos la información sobre las casas con más de una planta. Para ello, seleccionaremos todas las celdas que contengan la palabra 'plantas' (en plural), y de allí extraeremos el número por medio de una expresión regular:

In [ ]:
f_c = (df['floor'][df.floor.str.contains('plantas', na=False)]
       .str.extract('(\d+)')
       .astype('float64')
       .rename(columns={0:'floor_count'})
      )

Una vez hemos extraído el número de plantas de las casas con más de una, actualizamos la variable:

In [ ]:
df.update(f_c)
print(df.floor_count.unique())

<br />

Por último modificaremos la variable <font color=#5F66A1>__floor__</font>, para que solo recoja el número de planta en el que se encuentra el piso:

In [ ]:
df['floor'] = (df['floor']
               .str.replace('(exterior|interior|1 planta)', '')
               .str.replace('(\d+) plantas', '')
               .str.strip()
               .str.replace('planta ', '')
               .replace('', np.nan)
               .astype('category')
              )

print(df.floor.unique())

Vemos que hay 30 categorías distintas, lo cual parece excesivo para diferenciar distitos números de planta. Por ello, exploramos si podemos colapsar ciertas categorías.

In [ ]:
df.groupby('floor').count().sort_values('house_id',ascending=False)['house_id']

En vista de la dispersión de viviendas en función de la planta, se opta por colapsar todas aquellas por encima de la 7ª. Además, se hará lo propio con las plantas por debajo del 1º (entreplantas, bajos, sótanos, ...). Finalmente, se observan casos como planta núm. 60ª y 30ª, las cuales parecen errores tipográficos, por lo que se corrigen como 6ª y 3ª respectivamente:

In [ ]:
df['floor'] = (df['floor']
               .str.replace('(bajo|entreplanta|semi-sótano|sótano|-1|-2)','<0ª')
               .str.replace('60ª','6ª')
               .str.replace('30ª','3ª')
               .str.replace(r'(24ª|23ª|22ª|21ª|20ª|19ª|18ª|17ª|9ª|8ª)', '>8ª')
               .str.replace(r'(16ª|15ª|14ª|13ª|12ª|11ª|10ª)', '>8ª')        
              )

print(df.floor.unique())

### <font color=#5F66A1>heating</font>

In [ ]:
print(df.heating.unique())

Sustraemos la palabra 'calefacción' para mayor interpretabilidad, y la definimos como categórica:

In [ ]:
df['heating'] = (df['heating']
                 .str.replace('calefacción ', '')
                 .astype('category')
                )

### <font color=#5F66A1>house_type</font>

In [ ]:
df['house_type'] = (df['house_type']
                    .str.strip()
                    .astype('category')
                   )

df.groupby('house_type').count().sort_values('house_id',ascending=False)['house_id']

Vemos nuevamente, que hay excesivas categorías. Colapsamos algunos:

In [ ]:
df['house_type'] = (df['house_type']
                    .str.replace(r'Alquiler .*','Alquiler')
                    .str.replace(' independiente','')
                    .str.replace('pareado','adosado')
                    .str.replace(r'(Caserón|Casa de pueblo|Casa terrera)','Casa rural')
                    .str.replace(r'(Palacio|Torre|Castillo|Cortijo)','Otros')
                   )
                    
print(df.house_type.unique())

### <font color=#5F66A1>orientation</font>

In [ ]:
print(df.orientation.unique())

No requiere estandarización salvo definirla como categórica:

In [ ]:
df['orientation'] = (df['orientation'].astype('category'))

## Estandarización de variables booleanas

Analizamos qué valores tienen las variables booleanas:

In [ ]:
print(f'Valores para {b}air_conditioner{n}: {df.air_conditioner.unique()}')
print(f'Valores para {b}balcony{n}: {df.balcony.unique()}')
print(f'Valores para {b}built_in_wardrobe{n}: {df.built_in_wardrobe.unique()}')
print(f'Valores para {b}chimney{n}: {df.chimney.unique()}')
print(f'Valores para {b}garden{n}: {df.garden.unique()}')
print(f'Valores para {b}kitchen{n}: {df.kitchen.unique()}')
print(f'Valores para {b}lift{n}: {df.lift.unique()}')
print(f'Valores para {b}reduced_mobility{n}: {df.reduced_mobility.unique()}')
print(f'Valores para {b}storage_room{n}: {df.storage_room.unique()}')
print(f'Valores para {b}swimming_pool{n}: {df.swimming_pool.unique()}')
print(f'Valores para {b}terrace{n}: {df.terrace.unique()}')
print(f'Valores para {b}unfurnished{n}: {df.unfurnished.unique()}')

Dado que las variables <font color=#5F66A1>__kitchen__</font> y <font color=#5F66A1>__unfurnished__</font> son variables ligadas exclusivamente a casas de alquiler, las eliminamos:

In [ ]:
df = df.drop(columns=['kitchen', 'unfurnished'])

Por otra parte, vemos que la variable <font color=#5F66A1>__lift__</font>, a pesar de ser booleana, contiene valores NaN, por lo que la definimos como categórica por ahora:

In [ ]:
df['lift'] = df['lift'].astype('category')

Finalmente, el resto de variables categóricas las definimos como booleanas:

In [ ]:
df['air_conditioner'] = df['air_conditioner'].astype('bool')
df['balcony'] = df['balcony'].astype('bool')
df['built_in_wardrobe'] = df['built_in_wardrobe'].astype('bool')
df['chimney'] = df['chimney'].astype('bool')
df['garden'] = df['garden'].astype('bool')
df['reduced_mobility'] = df['reduced_mobility'].astype('bool')
df['storage_room'] = df['storage_room'].astype('bool')
df['swimming_pool'] = df['swimming_pool'].astype('bool')
df['terrace'] = df['terrace'].astype('bool')

In [ ]:
print(f'Nuevos valores para {b}air_conditioner{n}: {df.air_conditioner.unique()}')
print(f'Nuevos valores para {b}balcony{n}: {df.balcony.unique()}')
print(f'Nuevos valores para {b}built_in_wardrobe{n}: {df.built_in_wardrobe.unique()}')
print(f'Nuevos valores para {b}chimney{n}: {df.chimney.unique()}')
print(f'Nuevos valores para {b}garden{n}: {df.garden.unique()}')
print(f'Nuevos valores para {b}lift{n}: {df.lift.unique()}')
print(f'Nuevos valores para {b}reduced_mobility{n}: {df.reduced_mobility.unique()}')
print(f'Nuevos valores para {b}storage_room{n}: {df.storage_room.unique()}')
print(f'Nuevos valores para {b}swimming_pool{n}: {df.swimming_pool.unique()}')
print(f'Nuevos valores para {b}terrace{n}: {df.terrace.unique()}')

## Estandarización de variables ligadas a la ubicación

La localización de cada vivienda viene especificada por medio de 4 variables distintas, las cuales están organizadas de forma jerárquica del siguiente modo:

+ <font color=#5F66A1>__loc_zone__</font> &nbsp; > &nbsp;<font color=#5F66A1>__loc_city__</font> &nbsp;> &nbsp;<font color=#5F66A1>__loc_district__</font> &nbsp;> &nbsp;<font color=#5F66A1>__loc_neigh__</font>

### <font color=#5F66A1>loc_zone</font>

In [ ]:
print(df.loc_zone.unique())

Definimos las provincias con las iniciales y convertimos la variable en categórica:

In [ ]:
df['loc_zone'] = (df['loc_zone']
                  .str.replace(', Vizcaya',' (BIZ)')
                  .str.replace(', Guipúzcoa',' (GIP)')
                  .astype('category')
                 )

print(f"Número de zonas: {len(df.loc_zone.unique())}")

### <font color=#5F66A1>loc_city</font>

In [ ]:
print(df.loc_city.unique()[:15])

Los valores están ya lo suficientemente estandarizados, por lo que los dejamos como están, y establecemos la variable como categórica:

In [ ]:
df['loc_city'] = df['loc_city'].astype('category')
print(f"Número de pueblos/ciudades: {len(df.loc_city.unique())}")

### <font color=#5F66A1>loc_district</font>

In [ ]:
print(f"Número de distritos: {len(df.loc_district.unique())}")

Vemos que hay {{len(df.loc_district.unique())}} distritos distintos. No obstante, en muchos casos, en vez del distrito como tal, el campo guarda información relativa a la calle, urbanización, etc., como puede verse a continuación:

In [ ]:
print(df.sample(frac=1, random_state=89)['loc_district'][df.loc_district.notna()].head(5))

Dado que cada distrito está precedido por la palabra __Distrito__, eliminamos todos los valores que no coincidan con este patrón:

In [ ]:
df['loc_district'] = (df['loc_district']
                      .str.extract(r'(Distrito .*)')[0]
                      .str.replace('Distrito ','')
                     )

Por otro lado, para asegurarnos de que no haya distritos distintos compartiendo un mismo nombre, agregaremos las iniciales de cada ciudad:

In [ ]:
df['loc_district'] = (np.where(df.loc_district.notna()
                      , df.loc_district+' ('+df.loc_city.str[:4]+')'
                      , np.nan)
                     )

print(df.sample(frac=1, random_state=89)['loc_district'][df.loc_district.notna()].head(5))

Finalmente, comprobamos el número de distritos que hay tras el proceso de limpieza:

In [ ]:
print(f"Número de distritos tras limpieza: {len(df.loc_district.unique())}")

### <font color=#5F66A1>loc_neigh</font>

In [ ]:
print(f"Número de barrios: {len(df.loc_neigh.unique())}")

Vemos que hay {{len(df.loc_neigh.unique())}} barrios distintos. Sin embargo, sucede lo mismo que sucedía con el distrito, que en vez del barrio, en muchos casos el campo guarda información relativa a la calle, urbanización, etc.:

In [ ]:
print(df.sample(frac=1, random_state=89)['loc_neigh'][df.loc_neigh.notna()].head(5))

En este caso, el barrio viene precedido siempre por la palabra __Barrio__. Así, repetimos el mismo proceso que en el anterior caso. No obstante, en esta ocasión limitamos la extracción de barrios solo a ciertos pueblos o ciudades:

In [ ]:
city_filt = ['Bilbao', 'Leioa', 'Getxo', 'Donostia/San Sebastián']

df['loc_neigh'] = (df['loc_neigh']
                      .str.extract(r'(Barrio .*)')[0]
                      .str.replace('Barrio ','')
                     )

df['loc_neigh'] = (np.where(df.loc_neigh.notna()
                      , df.loc_neigh+' ('+df.loc_city.str[:4]+')'
                      , np.nan)
                  )
df['loc_neigh'] = (np.where(np.isin(df.loc_city, city_filt)
                      , df.loc_neigh
                      , np.nan)
                  )

print(df.sample(frac=1, random_state=89)['loc_neigh'][df.loc_neigh.notna()].head(5))

Finalmente, comprobamos el número de barrios que hay tras el proceso de limpieza:

In [ ]:
print(f"Número de barrios tras limpieza: {len(df.loc_neigh.unique())}")

### <font color=#5F66A1>location</font>

Por último, dado que no todas las viviendas tienen un valor para  <font color=#5F66A1>__loc_district__</font> o <font color=#5F66A1>__loc_neigh__</font>, vamos a generar una variable que incluya la información más específica posible por cada vivienda.

Así, en aquellos casos que se especifique el barrio, almacenara el barrio, sino, la del distrito, y en caso de que ninguna de las dos se indicara, la del pueblo: 

In [ ]:
df['location'] = df['loc_neigh']

df['location'] = (np.where(df.location.isna()
                      , df.loc_district
                      , df.location)
                 )

df['location'] = (np.where(df.location.isna()
                      , df.loc_city
                      , df.location)
                 )

print(df.sample(frac=1, random_state=89)['location'].head(5))
print(f"\nNúmero de ubicaciones distintas: {len(df.location.unique())}")

Por último, dado que ya nos las necesitamos, eliminamos las variables <font color=#5F66A1>__loc_district__</font> y <font color=#5F66A1>__loc_neigh__</font>:

In [ ]:
df = df.drop(columns=['loc_district', 'loc_neigh'])

# Verificación de los datos

## Descarte de las casas de alquiler

En el conjunto de datos que disponemos, se mezclan anuncios con casa en venta y alquiler. Este análisis se centra en la venta de casas, por lo tanto, los anuncios ligados al alquiler han de descartarse.

Para ello, la forma más sencilla es utilizar la variable <font color=#5F66A1>__house_type__</font>:

In [ ]:
h_num = len(df[df['house_type'] == 'Alquiler'])
df = df[df['house_type'] != 'Alquiler']
print(f'Se han descartado {h_num} casas de alquiler')

## Verificación de casas duplicadas

Lo primero que haremos será verificar que no haya anuncios duplicados. Para ello, dado que disponemos de un identificador de anuncio único, comprobaremos que no haya más de una vivienda con este id:

In [ ]:
h_num = df[df.house_id.duplicated()].shape[0]
df.drop_duplicates(subset='house_id', inplace=True)
print(f'Se han descartado {h_num} casas duplicadas del dataset')

Sin embargo, muchas veces una misma vivienda se ofrece en varias inmobiliarias, y cada una de ellas, genera un anuncio distinto, por lo que con solo comprobar el id del anuncio no basta.

Detectar este tipo de viviendas duplicadas es esencial. Pero esta vez no disponemos de un campo que nos facilite la tarea, por lo que tendremos que generar un filtro manual basado en coincidencias que identifique estas viviendas mediante estrategias heurísticas:

Así, dado que la gran mayoría de anuncios con viviendas repetidas corresponden a viviendas con precios elevados, definiremos una función que, para cada vivienda a partir de los 400.000€, vea si hay otra con exactamente el mismo precio en la misma zona, y que además, tenga al menos una serie características en común.

El filtro lo desarrollaremos asignando pesos a los diferentes atributos, de modo que algunos tengan más importancia a la hora de establecer si son lo suficientemente parecidas como para considerarlas la misma. Así, afinando estos pesos, seremos más o menos permisivos con ciertos aspectos.

En este caso, los pesos se han ajustado el filtro para que distinga lo mejor posible si son iguales. No obstante, dada la imposibilidad de ser 100% certeros en esta tarea, habrá viviendas que no sean iguales y que sin embargo las tratemos como tal. Se asume ese error a falta de una solución mejor:

In [ ]:
import numbers

''' checks if 2 houses have minimal conditions to assume that could be equal'''
def check_equality_conditions(h1, h2):
    
    cond_price = h1.price == h2.price
    cond_location = h1.location == h2.location
        
    return cond_price & cond_location


''' calculates similarity score between 2 houses based on different weights'''
def get_similarity(h1, h2):
    
    # improve house type condition match
    h1.house_type = h1.house_type.replace('Chalet adosado','Casa o chalet')
    h1.house_type = h1.house_type.replace('Casa rural','Casa o chalet')
    h1.house_type = h1.house_type.replace('Dúplex','Piso')
    h1.house_type = h1.house_type.replace('Ático','Piso')
    h2.house_type = h2.house_type.replace('Chalet adosado','Casa o chalet')
    h2.house_type = h2.house_type.replace('Casa rural','Casa o chalet')
    h2.house_type = h2.house_type.replace('Dúplex','Piso')
    h2.house_type = h2.house_type.replace('Ático','Piso')
    
    weights = [{"atr":'m2_real', "weight":2, "penalty":-0.5, "w_margin":0.15, "p_margin":0.15},
               {"atr":'house_type', "weight":1.5, "penalty":-3, "w_margin":0, "p_margin":0},
               {"atr":'bath_num', "weight":1.5, "penalty":-0.5,"w_margin":0, "p_margin":0.15},
               {"atr":'room_num', "weight":1.5, "penalty":-0.5, "w_margin":0, "p_margin":0.2},
               {"atr":'floor', "weight":1.5, "penalty":-1, "w_margin":0, "p_margin":0},
               {"atr":'construct_date', "weight":3, "penalty":0, "w_margin":0, "p_margin":0},
               {"atr":'ground_size', "weight":2, "penalty":0, "w_margin":0.30, "p_margin":30},
               {"atr":'m2_useful', "weight":2, "penalty":0, "w_margin":0.15, "p_margin":0.15},
               {"atr":'garage', "weight":0, "penalty":-0.5, "w_margin":0, "p_margin":0},
               {"atr":'lift', "weight":0, "penalty":-2, "w_margin":0, "p_margin":0},
               {"atr":'condition', "weight":0, "penalty":-1, "w_margin":0, "p_margin":0},
               {"atr":'swimming_pool', "weight":0, "penalty":-2, "w_margin":0, "p_margin":0}]
               
    score = 0
    for w in weights:
        
        if w['w_margin'] == 0 and w['p_margin'] == 0:
            if h1[w['atr']]==h2[w['atr']]:
                score += w['weight']
            else:
                score += w['penalty']
        
        else:
            if (h1[w['atr']] <= (h2[w['atr']] + h2[w['atr']]* w['w_margin'])
                and h1[w['atr']] >= (h2[w['atr']] - h2[w['atr']]* w['w_margin'])):
                score += w['weight']
            elif (h1[w['atr']] <= (h2[w['atr']] + h2[w['atr']]* w['p_margin'])
                and h1[w['atr']] >= (h2[w['atr']] - h2[w['atr']]* w['p_margin'])):
                score = score
            else:
                score += w['penalty']
    
    #print(\nscore)
    #print('https://www.idealista.com/inmueble/'+str(h1.house_id))
    #print('https://www.idealista.com/inmueble/'+str(h1.house_id))
    return score


''' calculates how much information is contained in the ad'''
def get_info_score(h):
    
    info_score = (- h.isna().sum()  # inverse count of NaN's
                  + h.garden + h.air_conditioner + h.balcony + h.chimney
                  + h.built_in_wardrobe + h.storage_room + h.swimming_pool)
    
    return info_score



''' MAIN FUNCTION: returns less informative duplicated houses indexes'''
def get_duplicated(price_bound=400000):
    h_to_del=[]
    
    for i,row1 in df[df['price']>price_bound].iterrows():
        for j,row2 in df[df['price']>price_bound].iterrows():

            if i == j:
                continue 

            # check if houses are already marked for delete
            if ((i in h_to_del) or (j in h_to_del)):
                continue 

            # check if houses fit minimal equality conditions
            equal_cond = check_equality_conditions(row1,row2)
            if not equal_cond:
                continue 
    
            # check if 2 houses are the same
            similarity_score = get_similarity(row1,row2)
            if similarity_score >= 0:
                
                # check which ad has more information ; KEEP THAT
                info_score1 = get_info_score(row1)
                info_score2 = get_info_score(row2)

                # delete less informative duplicate house
                if info_score1>=info_score2:
                    h_to_del.append(j)
                else:
                    h_to_del.append(i)
                    
    return h_to_del

Una vez definida la función que filtra las casas duplicadas, la ejecutamos:

In [ ]:
#h_to_delete = get_duplicated(400000);

# save the list in a pickle3
#pickle.dump(h_to_delete, open('./processed_data/h_to_delete.p', "wb" ));

h_to_delete = pickle.load(open('./processed_data/h_to_delete.p',"rb"))
print(f"Se han encontrado {len(h_to_delete)} casas duplicadas");

Finalmente, descartamos del dataset las casas detectadas por nuestro filtro:

In [ ]:
df = df.drop(h_to_delete)

# Gestión de ruido y valores extremos

En este apartado cotejaremos que los valores estén dentro de un rango plausible. De este modo, por cada variable numérica, gestionaremos tanto los valores extremos como las incongruencias. Además, se cotejará también la veracidad de ciertas variables categóricas. 

## Ruido y outliers en variables numéricas

### <font color=#5F66A1>m2_real</font> & <font color=#5F66A1>m2_useful</font>

Empezemos mostrando en un boxplot la dispersión de los metros cuadadros reales de las viviendas:

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'m2_real']
 .dropna()
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Observamos que hay 2 casos muy extremos. Los exploramos a fin de determinar cómo tratarlos:

In [ ]:
df.query('m2_real > 50000').style.set_properties(subset=['ad_description'], **{'min-width': '800px'})

En la descripción del anuncio se ve que en un caso han sumado 3 ceros de más a la superficie real, y en el otro, han asignado el valor de la párcela. Por lo tanto, dado que en la descripción se detalla el tamaño real, se corrigen manualmente, y se vuelve a representar el boxplot:

In [ ]:
df['m2_real'][df.house_id == 82667064] = 415
df['m2_real'][df.house_id == 39173329] = 300

f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'m2_real']
 .dropna()
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Pese a haber resuelto los casos más extremos, vemos que aún siguen existiendo valores muy alejados del resto. De modo que volvemos a analizarlos a fin de determinar qué tratamiento darles:

In [ ]:
out_m2 = len(df[df['m2_real'] >1500])
(df
 .query('m2_real > 1500')
 .sort_values('m2_real',ascending=False)
 .head(5)
 .style.set_properties(subset=['ad_description'], **{'min-width': '1000px'})
)

Observamos que hay {{out_m2}} viviendas con precios superiores a los 1.500 $m^2$ reales, entre las cuales hay un convento, un camping, una nave industrial... este tipo de edificios quedan fuera de nuestro objeto de estudio y podrían generar distorsión, por lo que se eliminan. Por otro lado, encontramos nuevamente viviendas que tienen mal definida su superficie; éstas las corregimos de forma manual mediante la información contenida en otros campos:

In [ ]:
df['m2_real'][df.house_id == 82012713] = 90
df['m2_real'][df.house_id == 39733981] = 46

h_num = len(df[df.m2_real > 1500])
df = df[df.m2_real < 1500]
print(f'Se han descartado {h_num} viviendas')

Por otro lado, en el otro extremo, exploramos si hay viviendas con valores de superficie nulos o negativos: 

In [ ]:
cnt = len(df.query('m2_real < 1'))
print(f"Hay {cnt} viviendas con superficie nula o negativa")

Una vez hemos gestionado los valores extremos de la variable <font color=#5F66A1>__m2_real__</font>, hacemos lo propio con <font color=#5F66A1>__m2_useful__</font>:

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'m2_useful']
 .dropna()
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Vemos que el valor más extremo se encuentra en los 1.200 $m^2$, lo cual entra dentro del rango plausible tras haber eliminado las casas con más de 1.500 $m^2$ reales, por lo que en este caso, no se realiza ninguna acción más, y se procede a verificar que no haya valores nulos o negativos:

In [ ]:
cnt = len(df.query('m2_useful < 1'))
print(f"Hay {cnt} viviendas con superficie nulas o negativa")

Por último, los metros útiles de una vivienda han de ser inferiores a sus metros reales. Verificamos si hay algún caso que no cumpla este criterio:

In [ ]:
cnt = len(df.query('m2_real < m2_useful'))
print(f"Hay {cnt} viviendas con incongruencias entre las superficies")

Además, por lo general, los metros útiles se situan en torno a un 0%-20% por debajo de los metros reales. Por ello, exploremos gráficamente esta relación para hallar valores extremos:

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(12, 6), sharex=True)
sns.despine(left=True)

(df
 .pipe((sns.scatterplot, 'data'),x="m2_real", y="m2_useful", color=clr['pr'], legend="full")
)

plt.plot([0,1200], [0, 1200], '-.', color='orange', linewidth = 2)
plt.plot([80,1200], [0, 960], '-.', color='orange', linewidth = 2)
plt.tight_layout()

Comprobamos que, en efecto, la mayoría de viviendas tienen una relación esperada entre metros de construcción y metros útiles. No obstante, encontramos algunas viviendas con muy pocos metros útiles para los metros reales que tienen. En concreto, encontramos 2 muy alejadas. Para cotejar si son plausibles, las exploremos más a fondo:

In [ ]:
df['m2_relation'] = df['m2_real'] - df['m2_useful']

(df
 .sort_values('m2_relation', ascending=False)[:2]
 .style.set_properties(subset=['ad_description'], **{'min-width': '1400px'})
)

Vemos que en el primer caso, se ha asigando un 0 de menos a la variable de metros útiles, y en el segundo, se han asignado los metros de la parcela a la variable de metros reales. Por lo tanto, se corrigen:

In [ ]:
df = df.drop(columns='m2_relation')
df['m2_useful'][df.house_id == 85089573] = 740
df['m2_real'][df.house_id == 84234935] = 130

### <font color=#5F66A1>ground_size</font>

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(12, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'ground_size']
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Se observa como hay valores muy extremos, con uno especialmente alejado del resto. Analizamos los más extremos:

In [ ]:
(df.query('ground_size > 160000')
 .style.set_properties(subset=['ad_description'], **{'min-width': '1600px'}))

Vemos que, en principio, los valores son plausibles, por lo que no efectuamos ninguna tarea al respecto. Por otro lado, verificamos que no hay parcelas con valores negativos:

In [ ]:
cnt = len(df.query('ground_size < 0'))
print(f"Hay {cnt} viviendas con parcela nula o negativa")

### <font color=#5F66A1>price</font>

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(12, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'price']
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Vemos que la vivienda más cara de nuestro conjunto de datos está por encima de los 5 millones de euros. Si bien es muy cara, es plausible que nos encontremos con alguna de tal precio. Aún así, exploramos las viviendas con precios más elevados a fin de cotejar que los datos son correctos:

In [ ]:
(df
 .query('price > 3500000')
 .sort_values('price', ascending=False)
 .style.set_properties(subset=['ad_description'], **{'min-width': '2400px'})
)

Vemos que los precios parecen correctos, por lo que no realizamos ninguna acción al respecto. Por otro lado, verificamos si hay viviendas con precios excesivamente bajos:

In [ ]:
(df
 .query('price < 20000')
 .style.set_properties(subset=['ad_description'], **{'min-width': '600px'})
)

Vemos que las viviendas tienen bien asignado el precio, por lo que optamos por no descartarlos, entendiendo que forman parte también del conjunto de datos de casas en venta que queremos estudiar.

### <font color=#5F66A1>bath_num</font>

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(12, 3), sharex=True)
sns.despine(left=True)

def print_text(nums):  
    for num in nums:
        cnt = len(df.query('bath_num == @num'))
        plt.text(num-0.1, 1.008, cnt, size='medium', weight='semibold')

(df
 .groupby('bath_num')
 .count()
 .reset_index()
 .pipe((sns.scatterplot, 'data'), x='bath_num', y=1, size='house_id', sizes=(40, 3000)
       , color=clr['pr'], alpha=.7, legend=False))

ax.set_xticks(range(0,18))
print_text([0,7,8,9,10,11,12,15,17])

plt.setp(ax, yticks=[])
plt.tight_layout()

Vemos que hay unas pocas viviendas con un número muy elevado de baños. Las exploramos:

In [ ]:
(df
 .query('bath_num > 10')
 .sort_values('bath_num', ascending=False)
 .style.set_properties(subset=['ad_description'], **{'min-width': '1600px'})
)

Vemos que se tratan de hoteles, por lo que optamos por descartarlos del conjunto de datos por no ajustarse a nuestro objeto de estudio:

In [ ]:
del_count = len(df[df.bath_num > 10])
df = df[df.bath_num < 11]
print(f"Se han eliminado {del_count} viviendas")

Por último, vemos que hay bastantes viviendas sin baños. Exploramos para entender a que se debe:

In [ ]:
(df
 .query('bath_num == 0')
 .sort_values('bath_num', ascending=False)[:5]
 .style.set_properties(subset=['ad_description'], **{'min-width': '1600px'})
)

Vemos que el hecho de que las viviendas no dispongan de baños se debe a que se trata de viviendas que han de reformarse, lo cual tiene sentido. Por ello, verificamos que todas las viviendas sin baños sean en efecto viviendas que han de reformarse:

In [ ]:
len(df.query('bath_num == 0 & condition != "para reformar"'))

### <font color=#5F66A1>room_num</font>

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(12, 3), sharex=True)
sns.despine(left=True)

def print_text(nums):  
    for num in nums:
        cnt = len(df.query('room_num == @num'))
        plt.text(num-0.1, 1.008, cnt, size='medium', weight='semibold')

(df
 .groupby('room_num')
 .count()
 .reset_index()
 .pipe((sns.scatterplot, 'data'), x='room_num', y=1, size='house_id', sizes=(40, 3000)
       , color=clr['pr'], alpha=.7, legend=False))

ax.set_xticks(range(0,21))
print_text([0, 10,11,12,14,20])

plt.setp(ax, yticks=[])
plt.tight_layout()

En este caso, se observa también que hay algunas viviendas con un gran número de habitaciones. Las exploramos:

In [ ]:
(df
 .query('room_num > 12')
 .sort_values('room_num', ascending=False)
 .style.set_properties(subset=['ad_description'], **{'min-width': '1200px'})
)

En lo que respecta a las viviendas con 20 habitaciones, al no poder verificar si son datos correctos, se opta por eliminarlos. En el caso de las viviendas con 14 habitaciones, sin embargo, se comprueba que el numero elevado de estancias se debe a que son caserones, así que se mantienen:

In [ ]:
del_count = len(df[df.room_num > 15])
df = df[df.room_num < 15]
print(f"Se han eliminado {del_count} viviendas")

Finalmente, de forma análoga al estudio de la variable <font color=#5F66A1>__bath_num__</font>, se verifica que las viviendas con 0 habitaciones corresponden a casas por reformar:

In [ ]:
(df
 .query('room_num == 0 & condition!="para reformar"')
 .sort_values('room_num', ascending=False)[:5]
 .style.set_properties(subset=['ad_description'], **{'min-width': '1600px'})
)

Vemos que hay 5 viviendas sin habitaciones que no son para reformar. Sin embargo, corresponden a estudios, lo cual también tiene sentido, así que se mantienen. 

## Ruido y outliers en variables cateógricas

### <font color=#5F66A1>floor</font>

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'floor']
 .dropna()
 .pipe(sns.countplot, color=clr['pr'], alpha=0.7
       , order=['<0ª','1ª','2ª','3ª','4ª','5ª','6ª','7ª','>8ª']))

plt.setp(axes, yticks=[])
plt.tight_layout()

Vemos que la distribución de las plantas de las viviendas se ajusta a valores plausibles, por lo que las dejamos tal cual.

# Gestión de datos incompletos

En este apartado gestionaremos los valores vacíos que aún presenta el conjunto de datos.

## Estudio general de *missing values*

Empezamos analizando cómo estan repartidos nuestros *missing values*:

In [ ]:
for col in df.columns:
    na_count = sum(pd.isna(df[col]))
    na_perc = round((na_count*100)/len(df),1)
    if(na_count > 0 and na_perc < 50):
        print(f"- {b}{col}{n}: {na_perc}%")
    elif(na_count > 0 and na_perc >= 50):
        print(f"- {b}{col}{n}: {o}{na_perc}%{n}")

Vemos que hay muchas variables con *missing values*, algunas incluso con más del 50% de sus valores. Trataremos todas, excepto la descripción de los anuncios, ya que solo la usaremos como ayuda en la fase exploratoria.

## Imputación de variables numéricas

En el caso de <font color=#5F66A1>__ground_size__</font>, la ausencia de valores se asume que corresponde, en una amplia mayoría, a que la vivienda no tenga parcela. Por lo tanto, se opta por asignar a todos los valores faltantes un 0: 

In [ ]:
df['ground_size'] = (np.where(df['ground_size'].isna()
                        ,0
                        ,df['ground_size']))

Por otro lado, en el caso de <font color=#5F66A1>__m2_useful__</font>, dado el gran número de casos con NAs, y dada la la estrecha correlación con la variable <font color=#5F66A1>__m2_real__</font>, se opta directamente por descartarla:

In [ ]:
df = df.drop(columns='m2_useful')

Finalmente, respecto a la variable <font color=#5F66A1>__construct_date__</font>, debido a que casi un 70% de las viviendas no incluye esta información, se opta también por descartarla:

In [ ]:
df = df.drop(columns='construct_date')

## Imputación de variables categóricas

Hay ciertas variables en las que la ausencia de valor corresponde a un valor concreto. En el caso de <font color=#5F66A1>__lift__</font>, por ejemplo, los NAs corresponden a viviendas sin ascensor, por lo que realizamos la imputación de forma directa:

In [ ]:
df['lift'] = (np.where(df['lift'].isna()
                        ,0
                        ,df['lift'])).astype('bool')

En el caso de <font color=#5F66A1>__garage__</font>, la ausencia de valores corresponde con viviendas sin garaje. No obstante, la variable muestra el precio del garaje, y los valores nulos significan que el garaje se ofrece sin coste añadido. Por lo tanto, no podemos imputar con ceros la ausencia de garajes, ya que daríamos a entender justo lo contrario de lo que significa. Por ello, se opta por categorizar la variable afín de poder representar cada caso de forma adecuada:

In [ ]:
df['garage'] = (np.where(df['garage']<1000
                    ,'incluido en precio'
                    ,np.where(df['garage']>=1000
                        ,'pagando'
                        ,'sin garaje')))

df['garage'] = df['garage'].astype('category')

....... ACABAR imputación avanzada de:  indoor  / floor_count  /  condition  /  floor

... descartar: energetic_certif /  orientation  /  heating

# Estudio de duplicidades

Finalmente, hacemos un breve estudio sobre las duplicidades de las variables:

In [ ]:
n_records = len(df)
def duplicados_por_columna(df):
    for column in df:
        count_value = df[column].value_counts()
        common = count_value.iloc[0]
        rare = count_value.iloc[-1]
        if ((common / (1.0 * n_records)) > 0.7):
            print("Column:{} ## Common: {}% <> Rare: {}% ## Type: {}".format( df[column].name, 
                                       round(common / (n_records) * 100, 2), 
                                       round(rare / (n_records) * 100, 2),
                                       df[column].dtype
        ))
            
duplicados_por_columna(df)